In [3]:
import nltk
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec, KeyedVectors

## 훈련 데이터 다운로드

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ynebu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x17efb474908>)

- xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요
- 실질적 데이터는 영어문장으로만 구성된 내용을 담고 있는 <content>와 </content> 사이의 내
- 뿐만 아니라, <content>와 </content> 사이의 내용 중에는 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 등장하는데 이 또한 제거

## 훈련 데이터 전처리하기

In [12]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

In [13]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [14]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## Word2Vec 훈련시키기

#### Word2Vec의 하이퍼파라미터값
- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

In [9]:
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

Word2Vec는 입력한 단어에 대해서 가장 유사한 단어들을 출력하는 model.wv.most_similar을 지원

In [10]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8603622913360596), ('guy', 0.8051167726516724), ('lady', 0.759171187877655), ('boy', 0.756297767162323), ('girl', 0.7537803649902344), ('soldier', 0.7491384744644165), ('gentleman', 0.7403988838195801), ('poet', 0.7135505676269531), ('john', 0.6897315979003906), ('kid', 0.6871434450149536)]


## Word2Vec 모델 저장하고 로드하기

In [14]:
model.wv.save_word2vec_format('./eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [15]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8603622913360596), ('guy', 0.8051167726516724), ('lady', 0.759171187877655), ('boy', 0.756297767162323), ('girl', 0.7537803649902344), ('soldier', 0.7491384744644165), ('gentleman', 0.7403988838195801), ('poet', 0.7135505676269531), ('john', 0.6897315979003906), ('kid', 0.6871434450149536)]


## 한국어 Word2Vec 만들기
#### 훈련 데이터 전처리 하기

In [29]:
f_name = 'C:\\Users\\ynebu\\workspace\\github\\Data\\wikiextractor-master\\data\\AA\\wiki_00'
f = open(f_name, encoding="utf8")
# 데이터 확인
i=0
while True:
    line = f.readline()
    if line != '\n':
        i=i+1
        print("%d번째 줄 :"%i + line)
    if i==5:
        break 
f.close()

1번째 줄 :지미 카터

2번째 줄 :제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.

3번째 줄 :지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

4번째 줄 :1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.

5번째 줄 :1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.



Word2Vec을 위한 학습 데이터를 만들어보겠습니다.

In [39]:
from konlpy.tag import Okt  
okt=Okt()
fread = open(f_name, encoding="utf8")
# 파일을 다시 처음부터 읽음.
n=0
result = []

while True:
    line = fread.readline() #한 줄씩 읽음.
    if not line: break # 모두 읽으면 while문 종료.
    n=n+1
    if n%5000==0: # 5,000의 배수로 While문이 실행될 때마다 몇 번째 While문 실행인지 출력.
        print("%d번째 While문."%n)
    tokenlist = okt.pos(line, stem=True, norm=True) # 단어 토큰화
    temp=[]
    for word in tokenlist:
        if word[1] in ["Noun"]: # 명사일 때만
            temp.append((word[0])) # 해당 단어를 저장함

    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
        result.append(temp) # 결과에 저장
fread.close()

5000번째 While문.


In [40]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 2918


#### Word2Vec 훈련시키기

In [41]:
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [42]:
model_result1=model.wv.most_similar("대한민국")
print(model_result1)

[('한국', 0.9999167323112488), ('이후', 0.9999102354049683), ('일본', 0.9999037981033325), ('중', 0.9999001026153564), ('정부', 0.9998979568481445), ('후', 0.9998944997787476), ('당시', 0.999893069267273), ('등', 0.9998884797096252), ('국민', 0.9998880624771118), ('국가', 0.999887228012085)]


## 사전 훈련된 Word2Vec 임베딩(Pre-trained Word2Vec embedding)
자연어 처리 작업을 할때, 케라스의 Embedding()를 사용하여 갖고 있는 훈련 데이터로부터 처음부터 임베딩 벡터를 훈련시키기도 하지만, 위키피디아 등의 방대한 데이터로 사전에 훈련된 워드 임베딩(pre-trained word embedding vector)를 가지고 와서 해당 벡터들의 값을 원하는 작업에 사용 할 수도 있습니다.

#### 영어 Word2Vec Embbeding 
- 구글이 제공하는 사전 훈련된(미리 학습되어져 있는) Word2Vec 모델
- 구글은 사전 훈련된 3백만 개의 Word2Vec 단어 벡터들을 제공
- 임베딩 벡터의 차원은 300

In [21]:
# 구글의 사전 훈련된 Word2Vec 모델을 로드합니다.
model = KeyedVectors.load_word2vec_format('C:\\Users\\ynebu\\workspace\\github\\Data\\GoogleNews-vectors-negative300\\GoogleNews-vectors-negative300.bin', binary=True)  

In [22]:
print(model.vectors.shape) # 모델의 크기 확인

(3000000, 300)


In [23]:
print (model.similarity('this', 'is')) # 두 단어의 유사도 계산하기
print (model.similarity('post', 'book'))

0.40797037
0.057204388


In [24]:
print(model['book']) # 단어 'book'의 벡터 출력

[ 0.11279297 -0.02612305 -0.04492188  0.06982422  0.140625    0.03039551
 -0.04370117  0.24511719  0.08740234 -0.05053711  0.23144531 -0.07470703
  0.21875     0.03466797 -0.14550781  0.05761719  0.00671387 -0.00701904
  0.13183594 -0.25390625  0.14355469 -0.140625   -0.03564453 -0.21289062
 -0.24804688  0.04980469 -0.09082031  0.14453125  0.05712891 -0.10400391
 -0.19628906 -0.20507812 -0.27539062  0.03063965  0.20117188  0.17382812
  0.09130859 -0.10107422  0.22851562 -0.04077148  0.02709961 -0.00106049
  0.02709961  0.34179688 -0.13183594 -0.078125    0.02197266 -0.18847656
 -0.17480469 -0.05566406 -0.20898438  0.04858398 -0.07617188 -0.15625
 -0.05419922  0.01672363 -0.02722168 -0.11132812 -0.03588867 -0.18359375
  0.28710938  0.01757812  0.02185059 -0.05664062 -0.01251221  0.01708984
 -0.21777344 -0.06787109  0.04711914 -0.00668335  0.08544922 -0.02209473
  0.31835938  0.01794434 -0.02246094 -0.03051758 -0.09570312  0.24414062
  0.20507812  0.05419922  0.29101562  0.03637695  0.04

#### 한국어 Word2Vec Embbeding

In [25]:
model = Word2Vec.load('C:\\Users\\ynebu\\workspace\\github\\Data\\Ko_Word2Vec\\ko.bin')

In [26]:
result=model.wv.most_similar("강아지")
print(result)

[('고양이', 0.7290453314781189), ('거위', 0.7185634970664978), ('토끼', 0.7056223750114441), ('멧돼지', 0.6950401067733765), ('엄마', 0.693433403968811), ('난쟁이', 0.6806551218032837), ('한마리', 0.6770296096801758), ('아가씨', 0.675035297870636), ('아빠', 0.6729634404182434), ('목걸이', 0.6512461304664612)]


## 임베딩 벡터의 시각화(Embedding Visualization)
구글은 임베딩 프로젝터(embedding projector)라는 데이터 시각화 도구를 지원합니다. 모델이 저장되어져 있다면 아래 커맨드를 통해 시각화에 필요한 파일들을 생성할 수 있습니다.
- !python -m gensim.scripts.word2vec2tensor --input 모델이름 --output 모델이름
- eng_w2v_metadata.tsv와 eng_w2v_tensor.tsv 이 두 개 파일이 임베딩 벡터 시각화를 위해 사용할 파일입니다. 
- Load라는 버튼 클릭 후 위에 있는 Choose file 버튼을 누르고 eng_w2v_tensor.tsv 파일을 업로드하고, 아래에 있는 Choose file 버튼을 누르고 eng_w2v_metadata.tsv 파일을 업로드합니다.

In [15]:
!python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng_w2v

2020-07-25 13:22:42,673 - word2vec2tensor - INFO - running C:\Users\ynebu\Anaconda3\envs\test\lib\site-packages\gensim\scripts\word2vec2tensor.py --input eng_w2v --output eng_w2v
2020-07-25 13:22:42,673 - utils_any2vec - INFO - loading projection weights from eng_w2v
2020-07-25 13:22:44,775 - utils_any2vec - INFO - loaded (21613, 100) matrix from eng_w2v
2020-07-25 13:22:46,260 - word2vec2tensor - INFO - 2D tensor file saved to eng_w2v_tensor.tsv
2020-07-25 13:22:46,261 - word2vec2tensor - INFO - Tensor metadata file saved to eng_w2v_metadata.tsv
2020-07-25 13:22:46,263 - word2vec2tensor - INFO - finished running word2vec2tensor.py


#### 임베딩 프로젝터를 사용하여 시각화하기
구글의 임베딩 프로젝터를 사용해서 워드 임베딩 모델을 시각화해보겠습니다.
링크 : https://projector.tensorflow.org/